# Import Libraries

In [1]:
import utility_func as util
import gradio as gr

In [2]:
import warnings
warnings.filterwarnings('ignore')

# Functions

In [40]:
# Adjust plotting functions to accept axes as a parameter
def plot_ppm_variation(df, element, area):
    lat_list = util.np.sort(df['latitude'].unique())[::-1]
    n_plots = len(lat_list)

    if area == 'Ramagiri':
        # Create a figure with subplots
        fig, axes = util.plt.subplots(n_plots, 
                                      1, 
                                      figsize=(10, 2 * n_plots), 
                                      constrained_layout=True)
    elif area == 'Kodangal':
        # Create a figure with subplots
        fig, axes = util.plt.subplots(n_plots, 
                                      1, 
                                      figsize=(12, 2 * n_plots), 
                                      constrained_layout=True)
        
    if n_plots == 1:
        axes = [axes]  # Ensure axes is iterable
    
    for ax, lat in zip(axes, lat_list):
        subset = df[df['latitude'] == lat].sort_values(by='longitude')
        ax.plot(subset['longitude'], subset[element], marker='o', linestyle='-', color='b')
        ax.set_title(f'Concentration at Latitude {lat}')
        ax.set_xlabel('Longitude')
        ax.set_ylabel(f'Concentration(ppm)')
        ax.set_ylim(0, max(df[element]))
        ax.grid(True)

    # img_path = (util.os.path.join(curr_dir, 'Images')).replace('\\', '/')
    # combined_plot_path = img_path + "/combined_ppm_variation_plot.png"
    # fig.savefig(combined_plot_path)
    # util.plt.close(fig)
    
    return fig

# def plot_ppm_variation(df, element, area, lat, ax=None):
    
#     subset = df[df['latitude'] == lat].sort_values(by='longitude')
#     ax.plot(subset['longitude'], subset[element], marker='o', linestyle='-', color='b')
#     ax.set_title(f'Concentration at Latitude {lat}')
#     ax.set_xlabel('Longitude')
#     ax.set_ylabel(f'Concentration(ppm)')
#     ax.set_ylim(0, max(df[element]))
#     ax.grid(True)

def plot_contour(df, element, name, title, area, ax=None):
    if ax is None:
        ax = plt.gca()
    if element not in df.columns:
        raise KeyError(f"The element '{element}' does not exist in the dataframe.")
    
    min_lat_limit = min(df['latitude']) - 0.005
    max_lat_limit = max(df['latitude']) + 0.005
    min_long_limit = min(df['longitude']) - 0.005 
    max_long_limit = max(df['longitude']) + 0.005

    grid_x, grid_y = util.np.mgrid[
        min_long_limit:max_long_limit:200j,
        min_lat_limit:max_lat_limit:200j
    ]

    grid_z = util.griddata(
        (df['longitude'], df['latitude']),
        df[element],
        (grid_x, grid_y),
        method='cubic'
    )

    cmap = util.LinearSegmentedColormap.from_list("green_to_red", ["green", "yellow", "red"])
    
    if area == 'Ramagiri':
        ax.figure.set_size_inches(10, 6)
    elif area == 'Kodangal':
        ax.figure.set_size_inches(15, 5)

    cp = ax.contourf(grid_x, grid_y, grid_z, levels=15, cmap=cmap, alpha=0.7)
    util.plt.colorbar(cp, ax=ax, label=f'{name} concentration')
    
    cs = ax.contour(grid_x, grid_y, grid_z, levels=15, colors='k', linewidths=0.5)
    ax.clabel(cs, inline=True, fontsize=8, fmt='%1.0f')

    ax.set_title(title)
    ax.set_xlabel('Longitude')
    ax.set_ylabel('Latitude')
    ax.grid(True, color='gray', linestyle='--', linewidth=0.5)

# Function to get element symbol from the element name and area
def get_element_symbol(element_name, area):
    if area == 'Ramagiri':
        element = rg_elem_name_df.loc[rg_elem_name_df['Name'] == element_name, 'Symbol'].values[0]
    elif area == 'Kodangal':
        element = kg_elem_name_df.loc[kg_elem_name_df['Name'] == element_name, 'Symbol'].values[0]
    return element

# Function to update element dropdown options based on selected area
def update_elements(area):
    if area == 'Ramagiri':
        elements = list(rg_elem_name_df['Name'])
    elif area == 'Kodangal':
        elements = list(kg_elem_name_df['Name'])
    return gr.update(choices=elements)

# Function to be called by Gradio
def show_plots(element_name, area):
    df = DATASETS[area]

    element = get_element_symbol(element_name, area)

    img_path = (util.os.path.join(curr_dir, 'Images')).replace('\\', '/')

    combined_ppm_plot_path = plot_ppm_variation(df, element, area)

    contour_fig, contour_axes = util.plt.subplots()
    plot_contour(df, element, element_name, f'{area} - {element_name} Concentration contour map', area, ax=contour_axes)
    contour_fig_path = img_path + '/contour_plot.png'
    contour_fig.savefig(contour_fig_path)
    util.plt.close(contour_fig)
    
    return [contour_fig_path, combined_ppm_plot_path]

## Plotting ppm concentration plot on interface
def ppm_plot_interface(element_name, area):
    df = DATASETS[area]

    element = get_element_symbol(element_name, area)

    img_path = (util.os.path.join(curr_dir, 'Images')).replace('\\', '/')

    combined_ppm_plot_path = plot_ppm_variation(df, element, area)

    return combined_ppm_plot_path

# def ppm_plot_interface(element_name, area, lat):
#     lat_list = util.np.sort(ramagiri_df['latitude'].unique())[::-1]
    
#     if area == 'Ramagiri':
#         fig, ax = util.plt.subplots(figsize=(10, 2))
#     elif area == 'Kodangal':
#         fig, ax = util.plt.subplots(figsize=(12, 2))
#     ppm_plot = plot_ppm_variation(ramagiri_df, 'au', area, lat, ax=ax)
    
#     return ppm_plot

## Plotting ppm concentration plot on interface
def contour_plot_interface(element_name, area):
    df = DATASETS[area]

    element = get_element_symbol(element_name, area)

    img_path = (util.os.path.join(curr_dir, 'Images')).replace('\\', '/')
    
    contour_fig, contour_axes = util.plt.subplots()
    plot_contour(df, element, element_name, f'{area} - {element_name} Concentration contour map', area, ax=contour_axes)
    # contour_fig_path = img_path + '/contour_plot.png'
    # contour_fig.savefig(contour_fig_path)
    # util.plt.close(contour_fig)
    return contour_fig

## Elements for prediction
def get_elem_pred(area):
    if area == 'Ramagiri':
        return ['Lithium', 'Arsenic', 'Gold']
    elif area == 'Kodangal':
        return ['Silver', 'Copper', 'Zinc']

# Load Data

In [5]:
curr_dir = (util.os.getcwd()).replace('\\', '/')
data_path = (util.os.path.join(curr_dir, 'Data')).replace('\\', '/')

In [6]:
kodangal_df = util.pd.read_csv(data_path + '/Kodangal dataset/clean_Kodangal_gcM.csv')
ramagiri_df = util.pd.read_csv(data_path + '/Ramagiri dataset/clean_stream_sediments_57F11.csv')

In [16]:
rg_elem_name_df = util.pd.read_csv(data_path + '/Kodangal dataset/Kodangal_element_names.csv')
kg_elem_name_df = util.pd.read_csv(data_path + '/Ramagiri dataset/Ramagiri_element_names.csv')

# Gradio interface

In [17]:
rg_elems = util.elem_list(ramagiri_df)
kg_elems = util.elem_list(kodangal_df)

In [18]:
rg_elem_name_df.head()

,Symbol,Name
0,sio2,SiO2
1,al2o3,Al2O3
2,fe2o3,Fe2O3
3,tio2,TiO2
4,cao,CaO


In [19]:
kg_elem_name_df.head()

,Symbol,Name
0,sio2,SiO2
1,al2o3,Al2O3
2,fe2o3,Fe2O3
3,tio2,TiO2
4,cao,CaO


In [20]:
DATASETS = {
    'Ramagiri' : ramagiri_df,
    'Kodangal' : kodangal_df
}

In [46]:
# Create Gradio interface with dynamic element dropdown
with gr.Blocks() as interface:
    gr.Markdown("<h1 style='text-align: center;'>Geochemical Data Visualization & Prediction App</h1>")

    with gr.Tab('Visualization'):
        with gr.Row():
            area = gr.Dropdown(choices=["Ramagiri", "Kodangal"], label="Area")
            elements = gr.Dropdown(label="Element")
    
        area.change(fn=update_elements, inputs=[area], outputs=[elements])
    
        with gr.Row():
            ppm_button = gr.Button("Generate ppm concentration variation Plots")
            contour_button = gr.Button("Generate Contour Map")
    
        with gr.Row():
            contour_output = gr.Plot()
    
        with gr.Row():
            ppm_output = gr.Plot()
            
        ppm_button.click(fn=ppm_plot_interface, inputs=[elements, area], outputs=[ppm_output])
        contour_button.click(fn=contour_plot_interface, inputs=[elements, area], outputs=[contour_output])

    with gr.Tab('Prediction'):
        with gr.Row():
            pred_area = gr.Dropdown(choices=["Ramagiri", "Kodangal"], label="Select Area for Prediction")
            target_element = gr.Dropdown(label="Target Element")

        with gr.Row():
            predict_button = gr.Button("Predict")

        with gr.Row():
            prediction_output = gr.Textbox(label="Prediction Result")

interface.launch()

Running on local URL:  http://127.0.0.1:7879

To create a public link, set `share=True` in `launch()`.


In [42]:
def get_element_input_fields(area):
    """ Return input fields for elements based on the selected area. """
    elements = get_elements_for_prediction(area)
    return [gr.Number(label=f"Enter ppm for {elem}", visible=False) for elem in elements]

def update_element_visibility(area, elements):
    """ Update visibility of element fields based on area selection. """
    visible_elements = get_elements_for_prediction(area)
    for elem_field in elements:
        elem_field.visible = (elem_field.label.split(" ")[-1] in visible_elements)
    return elements